## 📚 Lab 1.4 - Comparativo de Modelos de Embeddings (APIs Cloud)

### 🎯 Objetivo deste notebook

Neste laboratório, vamos **comparar modelos de embeddings de APIs cloud**:
- **OpenAI** - `text-embedding-3-small` e `text-embedding-3-large`
- **Google Gemini** - `gemini-embedding-001`

**O que vamos avaliar:**
1. ✅ **Dimensionalidade** - Tamanho dos vetores (384, 768, 1536, 3072)
2. ✅ **Qualidade semântica** - Capacidade de distinguir conceitos
3. ✅ **Tokens consumidos** - Contagem EXATA via APIs oficiais
4. ✅ **Custo real** - Cálculo baseado em tokens exatos (não estimados)
5. ✅ **Visualizações** - Gráficos de similaridade e custos

### 💡 Por que comparar APIs cloud?

**Vantagens das APIs:**
- ✅ Sem necessidade de infraestrutura (GPU, servidores)
- ✅ Escalabilidade automática
- ✅ Modelos state-of-the-art sempre atualizados
- ✅ Baixa latência (rede global otimizada)

**Desvantagens:**
- ❌ Custo por token (paga por uso)
- ❌ Dados enviados para nuvem (privacidade)
- ❌ Dependência de internet

### 🔧 Passo 1: Dependências

Instale as bibliotecas necessárias (já disponíveis no container Jupyter):

```bash
pip install -U openai numpy scikit-learn python-dotenv langchain-google-genai requests
```

**Bibliotecas usadas:**
- `openai` - SDK oficial da OpenAI
- `langchain-google-genai` - Integração Google com LangChain
- `requests` - Chamadas REST diretas (para obter metadados completos do Google)
- `numpy` - Cálculos de similaridade cosseno
- `dotenv` - Gerenciamento de API keys

**⚠️ Nota importante:** Não instale `google-generativeai` junto com `langchain-google-genai` (conflito de dependências). Usamos REST API direta quando precisamos de metadados completos.

In [1]:
import os
import csv
import numpy as np

from typing import List, Tuple, Optional
from openai import OpenAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from pathlib import Path
from dotenv import load_dotenv

e:\01-projetos\11-work\11.34-engenharia-vetorial\.venv\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.5) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


## 🔑 Passo 2: Configurar API Keys

As APIs cloud exigem **chaves de autenticação** para uso.

#### Como obter API keys:

**OpenAI:**
1. Acesse: https://platform.openai.com/api-keys
2. Crie uma conta (se não tiver)
3. Gere uma nova API key (começará com `sk-...`)
4. **Importante:** Copie imediatamente (só aparece uma vez!)

**Google Gemini:**
1. Acesse: https://aistudio.google.com/app/apikey
2. Faça login com conta Google
3. Clique em "Create API key"
4. Copie a chave gerada

#### Estrutura do arquivo `.env`:

Crie um arquivo `.env` na raiz do projeto:

```ini
OPENAI_API_KEY=sk-proj-xxxxxxxxxxxxxxxxxxxxx
GOOGLE_API_KEY=AIzaSyXXXXXXXXXXXXXXXXXXXXXXXXXXX
```

**Segurança:**
- ✅ Arquivo `.env` deve estar no `.gitignore` (não commitar!)
- ✅ Nunca compartilhe suas API keys publicamente
- ✅ Use variáveis de ambiente em produção

O código abaixo verifica se as chaves foram carregadas corretamente.

In [2]:
# 2) Configuração e carregamento do .env (simplificado)
env_path = Path.cwd().joinpath('..', '..', '.env').resolve()
if env_path.exists():
    load_dotenv(env_path)
    print(f'🔎 .env carregado -> {env_path.resolve()}')
else:
    print('⚠️  .env não encontrado. Defina as variáveis de ambiente manualmente.')


# Checar chaves (rótulos simples)
print('OPENAI_API_KEY set? ->', bool(os.getenv('OPENAI_API_KEY')))
print('GOOGLE_API_KEY set? ->', bool(os.getenv('GOOGLE_API_KEY')))

🔎 .env carregado -> E:\01-projetos\11-work\11.34-engenharia-vetorial\.env
OPENAI_API_KEY set? -> True
GOOGLE_API_KEY set? -> True


## 🛠️ Passo 3: Funções de Embedding

Vamos criar **wrappers simples** para as APIs de embeddings.

### Arquitetura das funções:

```python
openai_embedding(text) → (embedding_vector, token_count)
google_embedding(text) → (embedding_vector, token_count)
```

**Fluxo interno:**
1. Recebe texto (ex: "O gato é um animal doméstico")
2. Envia para API cloud via HTTP
3. API processa com modelo de embeddings
4. Retorna vetor + metadados (tokens, dimensões)

### Por que retornar tokens?

**Tokens são a unidade de cobrança das APIs:**
- OpenAI: $0.02 por 1M tokens (small), $0.13 por 1M tokens (large)
- Google: Free até limite, depois $0.15 por 1M tokens

💡 **Analogia:** É como um táxi - você paga pela "distância percorrida" (tokens processados), não pelo número de chamadas.

**Contagem exata vs estimada:**
- ✅ **EXATA** (oficial): Retornada pela API (usada neste notebook)
- ❌ **ESTIMADA** (heurística): ~4 chars = 1 token (imprecisa)

In [3]:

# OpenAI embedding wrapper
def openai_embedding(text: str, model: str = 'text-embedding-3-small', return_usage: bool = False) -> Tuple[List[float], Optional[int]]:
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    resp = client.embeddings.create(input=text, model=model)
    emb = resp.data[0].embedding
    if return_usage:
        usage = getattr(resp, 'usage', None)
        total_tokens = getattr(usage, 'total_tokens', None) if usage is not None else None
        return emb, total_tokens
    return emb, None


## 🔍 Implementação REST da API Google (Detalhes Técnicos)

A função `google_embedding` usa **duas abordagens** dependendo da necessidade:

##### Modo 1: `return_usage=True` (Tokens Exatos)

**Fluxo completo:**

```text
1. POST https://generativelanguage.googleapis.com/v1beta/models/{model}:embedContent
   ↓
2. Recebe: embedding + metadados (pode incluir tokenCount)
   ↓
3. Se tokenCount não vier, faz 2ª chamada:
   POST .../{model}:countTokens
   ↓
4. Retorna: (embedding, token_count_exato)
```

**Endpoints usados:**
- `:embedContent` - Gera embeddings + retorna metadados
- `:countTokens` - Conta tokens de um texto (fallback)

##### Modo 2: `return_usage=False` (Simples)

**Fluxo simplificado:**

```text
1. Usa GoogleGenerativeAIEmbeddings do LangChain
   ↓
2. Retorna apenas embedding (sem contagem de tokens)
```

### Por que usar REST direta?

| Aspecto | Google REST | OpenAI REST | LangChain |
|---------|-------------|-------------|-----------|
| **Token count** | ✅ Exato (embedContent/countTokens) | ✅ Exato (response.usage.total_tokens) | ❌ Nem sempre disponível |
| **Metadados** | ✅ Completos | ✅ Completos | ⚠️ Limitados (depende do wrapper) |
| **Simplicidade** | ❌ Mais código (requests + parsing) | ✅ SDK simplifica | ✅ Mais fácil (integração LangChain) |
| **Conflitos** | ✅ Nenhum | ✅ Nenhum | ⚠️ Possível com google-generativeai |

### Fallback robusto

Se a REST API falhar (rede, erro temporário):
```python
try:
    # Tenta REST API
    embedding = rest_api_call()
except:
    # Fallback para LangChain + estimativa
    embedding = langchain_call()
    tokens = estimate_tokens(text)  # ~1 token por 4 chars
```

### 📖 Documentação oficial:

- **embedContent**: https://ai.google.dev/api/rest/v1beta/models/embedContent
- **countTokens**: https://ai.google.dev/api/rest/v1beta/models/countTokens
- **Pricing**: https://ai.google.dev/gemini-api/docs/pricing#gemini-embedding

💡 **Dica:** Para casos simples, use LangChain. Para controle de custos preciso, use REST direta.

In [4]:

# Google embedding wrapper (retorna embedding e tokens exatos via REST API)
def google_embedding(text: str, model: str = 'gemini-embedding-001', return_usage: bool = True) -> Tuple[List[float], Optional[int]]:        
    """
    Gera embeddings usando Google Gemini.
    
    Quando return_usage=True, usa REST API direta para obter token count exato.
    Quando return_usage=False, usa LangChain para simplicidade.
    """
    if return_usage:
        # Usar REST API direta para obter metadados completos incluindo token count
        import requests
        
        api_key = os.getenv('GOOGLE_API_KEY')
        if not api_key:
            raise ValueError('GOOGLE_API_KEY não encontrada nas variáveis de ambiente')
        
        url = f'https://generativelanguage.googleapis.com/v1beta/models/{model}:embedContent?key={api_key}'
        
        headers = {
            'Content-Type': 'application/json'
        }
        
        payload = {
            'model': f'models/{model}',
            'content': {
                'parts': [{'text': text}]
            }
        }
        
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            
            data = response.json()
            
            # Extrair embedding
            embedding = data.get('embedding', {}).get('values', [])
            
            # Extrair token count (pode estar em diferentes locais dependendo da versão da API)
            token_count = None
            
            # Tentar extrair de metadata
            if 'metadata' in data:
                token_count = data['metadata'].get('tokenCount')
            
            # Fallback: usar a API de count tokens se não vier na resposta
            if token_count is None:
                count_url = f'https://generativelanguage.googleapis.com/v1beta/models/{model}:countTokens?key={api_key}'
                count_payload = {
                    'contents': [{'parts': [{'text': text}]}]
                }
                count_response = requests.post(count_url, headers=headers, json=count_payload)
                if count_response.status_code == 200:
                    count_data = count_response.json()
                    token_count = count_data.get('totalTokens')
            
            return list(embedding), token_count
            
        except requests.exceptions.RequestException as e:
            print(f'Erro na chamada REST API do Google: {e}')
            # Fallback para LangChain com estimativa
            emb = GoogleGenerativeAIEmbeddings(model=model).embed_query(text)
            estimated_tokens = len(text) // 4
            return list(emb), estimated_tokens
    else:
        # Usar LangChain quando não precisar de usage (mais simples)
        emb = GoogleGenerativeAIEmbeddings(model=model).embed_query(text)
        return list(emb), None


## 🧪 Passo 4: Experimento Prático

Vamos gerar embeddings para **3 frases cuidadosamente escolhidas** e medir similaridade.

### Dataset de teste:

| # | Frase | Conceito |
|---|-------|----------|
| 1 | "O gato é um animal doméstico" | Animais/Pets |
| 2 | "O gato é um felino de estimação" | Animais/Pets |
| 3 | "A programação é importante para engenheiros de software" | Tecnologia |

### O que esperamos?

✅ **Similaridade 1-2 ALTA** (>0.8)
- Ambas falam de **gatos** como **animais de estimação**
- Conceitos semanticamente idênticos
- Embeddings deveriam ser muito próximos

✅ **Similaridade 1-3 BAIXA** (<0.4)
- Conceitos completamente **distintos** (animais vs tecnologia)
- Sem relação semântica
- Embeddings deveriam ser distantes

💡 **Analogia:** É como testar se um detector de cores consegue distinguir:
- Vermelho vs Rosa (similar) → Deveria dar score alto
- Vermelho vs Azul (diferente) → Deveria dar score baixo
- **Grande diferença** = Detector funciona bem!

In [5]:
# Exemplo e comparação: gerar embeddings e calcular similaridade

def cosine_sim(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    return float(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))

texts = [
    'O gato é um animal doméstico',
    'O gato é um felino de estimação',
    'A programação é importante para engenheiros de software'
]

emb1 = emb2 = emb3 = None
backend = None


## 📐 Similaridade Cosseno - Fundamentos Matemáticos

A **similaridade cosseno** é a métrica mais usada para comparar embeddings. Ela mede o **ângulo entre dois vetores** no espaço multidimensional.

### Intuição visual:

```text
Espaço 2D (simplificado):

Vetor A (texto 1): ────────→ (direção: 30°)
Vetor B (texto 2): ───────→  (direção: 35°)  ← Ângulo pequeno = SIMILAR
Vetor C (texto 3):    ↓      (direção: 90°)  ← Ângulo grande = DIFERENTE
                      
Similaridade A-B: cos(5°) ≈ 0.996 (muito similar!)
Similaridade A-C: cos(60°) ≈ 0.5 (moderado)
Similaridade A-D: cos(90°) = 0.0 (perpendiculares, sem relação)
```

### Fórmula matemática:

$$
\text{cosine\_similarity}(v_1, v_2) = \frac{v_1 \cdot v_2}{\|v_1\| \times \|v_2\|}
$$

**Componentes:**
- $v_1 \cdot v_2$ = **Produto escalar** (dot product)
  - Soma dos produtos elemento a elemento: $\sum_{i} v_{1i} \times v_{2i}$
- $\|v_1\|$ = **Norma L2** do vetor 1
  - Comprimento do vetor: $\sqrt{\sum_{i} v_{1i}^2}$
- $\|v_2\|$ = **Norma L2** do vetor 2

### Interpretação dos valores:

| Valor | Ângulo | Significado | Exemplo |
|-------|--------|-------------|---------|
| **1.0** | 0° | Vetores idênticos | Mesma frase repetida |
| **0.9 - 0.99** | 0° - 26° | Muito similares | "Gato" vs "Felino" |
| **0.7 - 0.89** | 26° - 45° | Similares | "Cachorro" vs "Animal" |
| **0.5 - 0.69** | 45° - 60° | Moderadamente relacionados | "Cachorro" vs "Treinamento" |
| **0.3 - 0.49** | 60° - 73° | Pouco relacionados | "Cachorro" vs "Computador" |
| **0.0 - 0.29** | 73° - 90° | Sem relação | "Gato" vs "Matemática" |
| **0.0** | 90° | Perpendiculares | Sem relação alguma |
| **-1.0** | 180° | Opostos | "Quente" vs "Frio" |

### Por que cosseno é ideal para embeddings?

1. **Normalizado** ✅
   - Resultado sempre entre -1 e 1
   - Fácil de interpretar e comparar

2. **Invariante à magnitude** ✅
   - Divide pelas normas → ignora "tamanho" do vetor
   - Foca apenas na **direção semântica**
   - Textos longos e curtos comparáveis

3. **Eficiente computacionalmente** ✅
   - Operações vetoriais rápidas com NumPy/GPU
   - $O(n)$ onde $n$ é a dimensão do vetor

4. **Padrão da indústria** ✅
   - Usado por FAISS, Pinecone, Qdrant, Weaviate
   - Compatível com normalização L2 (otimização comum)

### Exemplo prático deste notebook:

Se os resultados mostrarem:
- **Sim(1,2) = 0.87** → "Gato doméstico" e "Felino de estimação" são muito similares ✅
- **Sim(1,3) = 0.32** → "Gato doméstico" e "Programação" têm pouca relação ✅
- **Diferença = 0.55** → Modelo distingue muito bem os conceitos! 🎯

💡 **Dica:** Sempre normalize vetores antes de armazenar no banco (usa produto escalar simples = mais rápido).

In [6]:

# Tentar OpenAI (pequeno) -> OpenAI (large) -> Google -> Erro amigável
openai_small_available = bool(os.getenv('OPENAI_API_KEY'))
openai_large_available = bool(os.getenv('OPENAI_API_KEY'))
google_available = bool(os.getenv('GOOGLE_API_KEY'))


In [7]:

# Função helper para rodar OpenAI e obter uso

def get_openai_embeddings(texts_list, model_name):
    embeddings = []
    total_tokens = 0
    for t in texts_list:
        emb, usage = openai_embedding(t, model=model_name, return_usage=True)
        embeddings.append(emb)
        if usage is not None:
            try:
                total_tokens += int(usage)
            except Exception:
                pass
    return embeddings, total_tokens if total_tokens else None


In [8]:
# Aqui vão ser armazenados os resultados com os embeddings gerados pelas APIs da OpenAI e Google

# Exemplo de dicionário para armazenar os resultados

# results = {
#    'openai_small': {'embeddings': [...], 'tokens': 123, 'dim': 1536},
#    'openai_large': {'embeddings': [...], 'tokens': 456, 'dim': 3072},
#    'google': {'embeddings': [...], 'tokens': 789, 'dim': 3072},
# }

results = {}

# OpenAI small
if openai_small_available:
    try:
        emb_small, tokens_small = get_openai_embeddings(texts, 'text-embedding-3-small')
        results['openai_small'] = {'embeddings': emb_small, 'tokens': tokens_small, 'dim': len(emb_small[0])}
    except Exception as e:
        print('Falha ao gerar embeddings OpenAI small:', e)


In [9]:
# OpenAI large
if openai_large_available:
    try:
        emb_large, tokens_large = get_openai_embeddings(texts, 'text-embedding-3-large')
        results['openai_large'] = {'embeddings': emb_large, 'tokens': tokens_large, 'dim': len(emb_large[0])}
    except Exception as e:
        print('Falha ao gerar embeddings OpenAI large:', e)


In [10]:
# Google embeddings
if google_available:
    try:
        # Gera os embeddings com a API REST para obter token count exato
        emb_google = [google_embedding(t, return_usage=True) for t in texts]
        
        # Armazenar resultados
        results["google"] = {
            "embeddings": [e[0] for e in emb_google],
            "tokens": [e[1] for e in emb_google],
            "dim": len(emb_google[0][0]),
        }
    except Exception as e:
        print("Falha ao gerar embeddings Google:", e)

In [11]:
# Mostrar resultados: dimensão, primeiros 5 valores e similaridades por backend
# Garantir que results_cost exista (pode ter sido criado em célula futura)
if 'results_cost' not in globals():
    results_cost = {}

for k, v in results.items():
    print('\nBackend:', k)
    print('Dimensão:', v['dim'])

    # Mapear provider e método
    if k.startswith('openai'):
        provider = 'OpenAI'
        method = 'SDK'
    elif k == 'google':
        provider = 'Google'
        method = 'REST'
    else:
        provider = 'Other'
        method = 'Unknown'

    print(f'Provider: {provider}  Method: {method}')

    # Exibir tokens (somar se for lista)
    tokens_value = v.get('tokens')
    if isinstance(tokens_value, list):
        total_tokens = sum(t for t in tokens_value if t is not None)
        print(f'Tokens (se disponível): {total_tokens} (total de {len(tokens_value)} chamadas)')
    else:
        print('Tokens (se disponível):', tokens_value)

    for i, e in enumerate(v['embeddings']):
        print(f' Embedding {i+1} primeiros 5:', e[:5])
        
    # calcular similaridade
    sim_12 = cosine_sim(v['embeddings'][0], v['embeddings'][1])
    sim_13 = cosine_sim(v['embeddings'][0], v['embeddings'][2])
    print(f' Similaridade 1-2: {sim_12:.4f}')
    print(f' Similaridade 1-3: {sim_13:.4f}')

    est_cost = results_cost.get(k)
    print(f' Est Cost USD (aproximado): {est_cost if est_cost is not None else "N/A"}')



Backend: openai_small
Dimensão: 1536
Provider: OpenAI  Method: SDK
Tokens (se disponível): 32
 Embedding 1 primeiros 5: [0.001428517745807767, -0.004510881379246712, -0.04022705927491188, 0.04195206239819527, 0.015516397543251514]
 Embedding 2 primeiros 5: [-0.012964113615453243, 0.009385946206748486, -0.061422210186719894, 0.0561358705163002, 0.017261510714888573]
 Embedding 3 primeiros 5: [0.019466428086161613, 0.027966158464550972, -0.0038620447739958763, -0.009738600812852383, 0.04860682040452957]
 Similaridade 1-2: 0.8135
 Similaridade 1-3: 0.1420
 Est Cost USD (aproximado): N/A

Backend: openai_large
Dimensão: 3072
Provider: OpenAI  Method: SDK
Tokens (se disponível): 32
 Embedding 1 primeiros 5: [-0.021800197660923004, 0.039917152374982834, -0.00277250399813056, 0.0035163466818630695, 0.010698671452701092]
 Embedding 2 primeiros 5: [-0.016434069722890854, 0.02808525785803795, -0.002878795610740781, -0.02375573106110096, 0.024798443540930748]
 Embedding 3 primeiros 5: [0.0003863

## 📊 Interpretando os Resultados Comparativos

Nesta seção, comparamos **3 modelos de embeddings** em múltiplas dimensões:

### Modelos testados:

| Modelo | Dimensões | Provedor | Otimizado para |
|--------|-----------|----------|----------------|
| **text-embedding-3-small** | 1536 | OpenAI | Velocidade e custo |
| **text-embedding-3-large** | 3072 | OpenAI | Máxima qualidade |
| **gemini-embedding-001** | 3072 | Google | Máxima qualidade |

### Métricas analisadas:

#### 1. **Dimensionalidade (dim)**

**O que é:** Número de componentes no vetor de embedding.

| Dimensões | Implicações |
|-----------|-------------|
| **768** | ✅ Leve, rápido, menor custo de armazenamento |
| **1536** | 🟡 Equilíbrio qualidade/performance |
| **3072** | 🔴 Máxima qualidade, mas 2x mais pesado |

💡 **Analogia:** É como resolução de imagem:
- 768 = HD (suficiente para maioria)
- 1536 = Full HD (ótimo equilíbrio)
- 3072 = 4K (máximo detalhe, mas arquivo grande)

#### 2. **Tokens (contagem exata)**

**O que é:** Unidade de processamento das APIs (palavras, subpalavras, caracteres).

**Fontes dos dados:**
- OpenAI: `response.usage.total_tokens` (oficial)
- Google: REST API `:embedContent` ou `:countTokens` (oficial)

**Por que importa:**
- 🔑 Determina o **custo exato** da operação
- 📊 Permite comparar eficiência de tokenização entre APIs
- 💰 Essencial para planejamento de budget

**Comparação típica:**

```text
Texto: "O gato é um animal doméstico" (7 palavras)

OpenAI: ~6 tokens (tokenização BPE otimizada¹)
Google: ~8 tokens (tokenização mais granular)
```

> ¹ Tokenização BPE otimizada é uma técnica que divide palavras em subunidades frequentes, reduzindo o número total de tokens para textos comuns.
> - BPE (Byte Pair Encoding) é uma técnica de tokenização por subpalavras: ela representa textos como tokens que podem ser caracteres, subpalavras ou combinações frequentes.
> - “BPE otimizada” significa que o vocabulário e as regras de merge foram ajustados para esse modelo específico, gerando:
>   - menos tokens por texto (melhor compressão),
>   - mais tokens inteiros relevantes (menos fragmentação),
>   - e geralmente resultados mais eficientes (custo e latência menores).
> 
> Em resumo: em vez de dividir cada palavra em muitos pedaços, o modelo usa pares/substrings mais “inteligentes” e frequentes para reduzir tokens




#### 3. **Primeiros 5 valores do embedding**

**O que é:** Amostra dos componentes do vetor.

**Para que serve:**
- ✅ Verificar distribuição dos valores (normalmente -1 a +1)
- ✅ Detectar anomalias (valores extremos, NaN)
- ✅ Comparar escala entre modelos

**Exemplo:**
```python
[0.0234, -0.1567, 0.0891, -0.0456, 0.1234]
```

#### 4. **Similaridade 1-2 vs 1-3**

**O que esperamos:**

```
| Sim(1,2) >> Sim(1,3) |
|   ↓           ↓      |
| ALTA       BAIXA     |
| (>0.8)     (<0.5)    |
```

**Interpretação:**

| Cenário | Sim(1,2) | Sim(1,3) | Diferença | Avaliação |
|---------|----------|----------|-----------|-----------|
| **Ideal** | 0.89 | 0.28 | 0.61 | 🟢 Excelente discriminação |
| **Bom** | 0.85 | 0.45 | 0.40 | 🟡 Boa discriminação |
| **Moderado** | 0.78 | 0.58 | 0.20 | 🟠 Discriminação fraca |
| **Ruim** | 0.72 | 0.68 | 0.04 | 🔴 Modelo não distingue conceitos |

### Como ler os resultados:

**Exemplo de saída:**
```
| Backend: openai_small                                                 |
| Dimensão: 1536                                                        |
| Tokens: 18                                                            |
| Embedding 1 primeiros 5: [0.0234, -0.1567, 0.0891, -0.0456, 0.1234]   |
| Embedding 2 primeiros 5: [0.0198, -0.1489, 0.0923, -0.0412, 0.1189]   |
| Embedding 3 primeiros 5: [-0.0876, 0.0234, -0.1123, 0.0567, -0.0234]  |
| Similaridade 1-2: 0.8932  ← ALTA ✅ (textos similares)                |
| Similaridade 1-3: 0.2134  ← BAIXA ✅ (textos diferentes)              |
```

**Análise:**
- ✅ Diferença de 0.68 = Excelente!
- ✅ Modelo captura semântica corretamente
- ✅ Pronto para uso em RAG/busca semântica

### Fatores que afetam similaridade:

1. **Qualidade do modelo**
   - Modelos maiores (3072 dim) tendem a ter melhor discriminação
   - Mas nem sempre: depende do treinamento

2. **Domínio do texto**
   - Modelos treinados em textos gerais funcionam bem para casos gerais
   - Domínios específicos (médico, legal) podem precisar fine-tuning

3. **Tamanho do texto**
   - Textos muito curtos (<5 palavras): mais difícil distinguir
   - Textos longos (>100 palavras): embeddings mais estáveis

### Quando um modelo é considerado bom?

✅ **Critérios de qualidade:**
- [ ] Similaridade 1-2 > 0.75 (conceitos similares agrupados)
- [ ] Similaridade 1-3 < 0.50 (conceitos distintos separados)
- [ ] Diferença > 0.30 (boa discriminação)
- [ ] Tokens compatíveis com budget (custo aceitável)
- [ ] Dimensões adequadas para infraestrutura (armazenamento/consulta)

### Comparação custo vs qualidade:

```text
Trade-off típico:

Qualidade Alta │  text-emb-3-large (3072 dim, $0.13/1M)
               │  gemini-emb-001   (3072 dim, free/baixo custo, depois $0.15/1M)
Qualidade Boa  │  text-emb-3-small (1536 dim, $0.02/1M)
               └────────────────────────────────────────
                 Custo Baixo    →    Custo Alto
```

💡 **Recomendação geral:**
- **Prototipagem**: text-embedding-3-small (rápido, barato)
- **Produção média escala**: gemini-embedding-001 (free tier generoso)
- **Produção alta qualidade**: text-embedding-3-large (máxima precisão)

## 💰 Preços Oficiais das APIs (Atualizado Dez/2025)

### Tabela de preços por modelo:

| Modelo | Dimensões | Preço por 1M tokens | Preço por 1K tokens | Provedor |
|--------|-----------|---------------------|---------------------|----------|
| **text-embedding-3-small** | 1536 | $0.020 | $0.00002 | OpenAI |
| **text-embedding-3-large** | 3072 | $0.130 | $0.00013 | OpenAI |
| **gemini-embedding-001** | 3072 | $0.150 * | $0.00015 | Google |

**\* Google oferece free tier generoso:**
- Primeiros 1M tokens/mês: **GRÁTIS** (permite a Google treinar seus modelos)
- Depois: $0.15 por 1M tokens

### Exemplo de cálculo de custo:

```python
Texto: "O gato é um animal doméstico"
Tokens: ~8 tokens (varia por tokenizador)

# OpenAI small
custo = (8 / 1_000_000) * 0.020 = $0.00000016

# OpenAI large  
custo = (8 / 1_000_000) * 0.130 = $0.00000104

# Google
custo = (8 / 1_000_000) * 0.150 = $0.00000120
```

### Comparação de custo por operação:

| Operação | Tokens | OpenAI small | OpenAI large | Google |
|----------|--------|--------------|--------------|--------|
| **1 frase curta** | ~8 | $0.0000002 | $0.0000010 | $0.0000012 |
| **1 parágrafo** | ~100 | $0.000002 | $0.000013 | $0.000015 |
| **1 documento** | ~1000 | $0.00002 | $0.00013 | $0.00015 |
| **10K documentos** | 1M | $0.02 | $0.13 | $0.15 |

### Análise custo-benefício:

**Cenário 1: Startup (baixo volume)**
```
✅ Recomendação: Google Gemini
📊 Volume: <1M tokens/mês
💰 Custo: $0 (free tier)
🎯 Qualidade: Boa (768 dim)
```

**Cenário 2: Empresa média (médio volume)**
```
✅ Recomendação: OpenAI text-embedding-3-small
📊 Volume: 10M-100M tokens/mês
💰 Custo: $0.20 - $2.00/mês
🎯 Qualidade: Excelente (1536 dim)
```

**Cenário 3: Enterprise (alto volume + máxima qualidade)**
```
✅ Recomendação: OpenAI text-embedding-3-large
📊 Volume: >100M tokens/mês
💰 Custo: ~$13/mês por 100M
🎯 Qualidade: State-of-the-art (3072 dim)
```

### Referências oficiais:

**OpenAI Pricing:**
- 🌐 https://platform.openai.com/docs/models
- 📖 Embedding models: https://platform.openai.com/docs/guides/embeddings

**Google AI Pricing:**
- 🌐 https://ai.google.dev/gemini-api/docs/pricing#gemini-embedding
- 📖 Free tier details: https://ai.google.dev/pricing

### 💡 Dicas para reduzir custos:

1. **Cache de embeddings** ✅
   - Salve embeddings já calculados
   - Evite processar o mesmo texto 2x

2. **Batch processing** ✅
   - Agrupe múltiplos textos em uma chamada
   - Reduz overhead de rede

3. **Chunking inteligente** ✅
   - Divida documentos longos em pedaços
   - Use apenas chunks relevantes para embedding

4. **Modelo híbrido** ✅
   - Use modelo pequeno para filtragem inicial
   - Aplique modelo grande apenas no top-k

5. **Monitoramento de uso** ✅
   - Configure alertas de gastos
   - Revise logs de consumo mensalmente

**⚠️ Importante:** Preços podem mudar. Sempre consulte documentação oficial antes de decisões de arquitetura.

In [12]:
# Estimativa de custos (usa preços públicos quando possível)
from math import ceil

PRICING = {
    'openai_text-embedding-3-small': {'per_1k_tokens_usd': 0.00002},
    'openai_text-embedding-3-large': {'per_1k_tokens_usd': 0.00013},
    'google_gemini-embedding-001': {'per_1k_tokens_usd': 0.00015}
}


## 🧮 Metodologia de Cálculo de Custos

### Como calculamos o custo EXATO:

Diferente de muitos tutoriais que usam **estimativas**, este notebook obtém **tokens reais** diretamente das APIs oficiais.

### Fontes de token count:

#### OpenAI (oficial):
```python
response = client.embeddings.create(input=text, model=model)
tokens = response.usage.total_tokens  # ← EXATO da API
```

**Campos disponíveis:**
- `prompt_tokens` - Tokens do input
- `total_tokens` - Total processado
- Retornado em **toda** chamada (sem custo adicional)

#### Google (oficial via REST):
```python
# Método 1: embedContent (retorna embedding + metadados)
POST /v1beta/models/{model}:embedContent
→ response.metadata.tokenCount  # ← EXATO

# Método 2: countTokens (apenas contagem)
POST /v1beta/models/{model}:countTokens
→ response.totalTokens  # ← EXATO
```

**Vantagens da abordagem REST:**
- ✅ Token count oficial (não estimado)
- ✅ Sem custo adicional pela contagem
- ✅ Mesmo tokenizador usado internamente

### Fórmula de custo:

$$
\text{Custo (USD)} = \frac{\text{Tokens}}{1000} \times \text{Preço por 1K tokens}
$$

**Exemplo prático:**
```python
# Dados reais do notebook
text = "O gato é um animal doméstico"
tokens = 8  # ← Retornado pela API OpenAI
price_per_1k = 0.00002  # text-embedding-3-small

cost = (8 / 1000) * 0.00002
     = 0.008 * 0.00002
     = $0.00000016
```

### Comparação: Tokens Exatos vs Estimados

| Método | Precisão | Como funciona | Quando usar |
|--------|----------|---------------|-------------|
| **Exato (API)** | ✅ 100% | Retornado pela API oficial | ✅ Sempre preferir |
| **Estimado (heurística)** | ❌ ~70-90% | 1 token ≈ 4 chars ou 1.3 palavras | Fallback apenas |

**Por que estimativas são imprecisas:**

```python
Texto: "OpenAI"

# Estimativa (1 palavra = 1.3 tokens)
estimated = 1 * 1.3 = 1.3 tokens

# Real (tokenizador BPE)
real = 2 tokens  # ["Open", "AI"]

# Erro: ~54% !
```

**Tokenizadores diferentes = contagens diferentes:**
- OpenAI usa **BPE** (Byte Pair Encoding)
- Google usa tokenizador próprio
- Mesma frase pode ter contagens distintas!

### Estrutura de dados do notebook:

```python
results = {
    'openai_small': {
        'embeddings': [[...], [...], [...]],  # 3 vetores
        'tokens': 18,                          # ← EXATO
        'dim': 1536
    },
    'google': {
        'embeddings': [[...], [...], [...]],
        'tokens': [7, 6, 8],  # ← EXATO (um por chamada)
        'dim': 768
    }
}
```

### Limitações e recomendações:

**Limitações:**
- ⚠️ Preços em `PRICING` dict devem ser **atualizados manualmente**
- ⚠️ APIs podem mudar estrutura de resposta (raro, mas possível)
- ⚠️ Free tiers têm limites (verificar quotas)

**Recomendações:**
1. ✅ **Use token count oficial** sempre que disponível
2. ✅ **Valide preços** periodicamente (links na célula anterior)
3. ✅ **Monitore consumo** com dashboards das plataformas:
   - OpenAI: https://platform.openai.com/usage
   - Google: https://console.cloud.google.com/billing
4. ✅ **Configure alertas** de gastos ($5, $10, $50)
5. ✅ **Documente custos** em produção para auditorias

### Fallback para estimativa:

```python
# Só usar se API não retornar tokens
def estimate_tokens_from_text(text: str) -> int:
    """
    Heurística: ~1.3 tokens por palavra
    Imprecisa, usar apenas como fallback!
    """
    words = text.split()
    return max(1, int(ceil(len(words) * 1.3)))
```

**Quando o fallback é acionado:**
- ❌ Erro de rede (timeout, API offline)
- ❌ API retorna resposta sem campo `usage` ou `tokenCount`
- ❌ Chave de API inválida (erro antes de retornar dados)

💡 **Dica:** Se você vê "usando estimativa" nos logs, investigue! Pode indicar problema na integração.

In [13]:
# Função para calcular custo baseado em tokens exatos

def estimate_tokens_from_text(text: str) -> int:
    """
    Heurística para estimar tokens quando não disponíveis da API.
    Assume ~1.3 tokens por palavra (aproximação).
    """
    words = text.split()
    return max(1, int(ceil(len(words) * 1.3)))


def calculate_cost(model_key: str, tokens: int) -> float:
    """
    Calcula o custo baseado em tokens exatos e preços definidos em PRICING.
    
    Args:
        model_key: Chave do modelo no dicionário PRICING
        tokens: Número exato de tokens processados
    
    Returns:
        Custo em USD
    """
    pricing = PRICING.get(model_key)
    if pricing is None:
        raise KeyError(f'Modelo {model_key} sem preço definido no PRICING dict')

    if 'per_1k_tokens_usd' not in pricing:
        raise KeyError(f'Modelo {model_key} sem estrutura de preço válida')
    
    return (tokens / 1000.0) * pricing['per_1k_tokens_usd']


# Calcular custo para os resultados do notebook (results dict)
results_cost = {}

for k, v in results.items():
    # Mapear nome do resultado para chave do modelo
    model_key: str | None = None
    if k == 'openai_small':
        model_key = 'openai_text-embedding-3-small'
    elif k == 'openai_large':
        model_key = 'openai_text-embedding-3-large'
    elif k == 'google':
        model_key = 'google_gemini-embedding-001'
    
    # Validar que temos o model_key antes de continuar
    if model_key is None:
        print(f'⚠️  Modelo desconhecido: {k} - pulando cálculo de custo')
        results_cost[k] = None
        continue

    tokens = v.get('tokens')
    cost = None
    
    try:
        if tokens is not None:
            # Para Google, tokens é uma lista (um por chamada); somar todos
            if isinstance(tokens, list):
                total_tokens = sum(t for t in tokens if t is not None)
            else:
                total_tokens = tokens
            
            # Calcular custo com tokens reais
            if total_tokens > 0:
                cost = calculate_cost(model_key, total_tokens)
        else:
            # Fallback APENAS se tokens não disponível (não deveria acontecer)
            print(f'⚠️  Aviso: tokens não disponível para {k}, usando estimativa')
            total_tokens = sum(estimate_tokens_from_text(t) for t in texts)
            cost = calculate_cost(model_key, total_tokens)
            
    except Exception as e:
        print(f'❌ Erro ao calcular custo para {k}: {e}')
        cost = None
    
    results_cost[k] = cost

# Mostrar custos calculados
print('\n💰 Custos por modelo (esta execução):')
print('=' * 50)
for k, c in results_cost.items():
    if c is not None:
        print(f'{k:20} → ${c:.8f} USD')
    else:
        print(f'{k:20} → [erro no cálculo]')

print('\n📌 Observação: Valores baseados em tokens REAIS das APIs.')
print('   Revise PRICING periodicamente para manter preços atualizados.')


💰 Custos por modelo (esta execução):
openai_small         → $0.00000064 USD
openai_large         → $0.00000416 USD
google               → $0.00000360 USD

📌 Observação: Valores baseados em tokens REAIS das APIs.
   Revise PRICING periodicamente para manter preços atualizados.


In [ ]:
# 6) Gerar relatório: CSV + gráficos + instruções para exportar slides/PDF


# tentativas de import para pandas/matplotlib; se faltar, gerar CSV somente
have_pandas = False
have_matplotlib = False
pd = None
plt = None
try:
    import pandas as pd
    have_pandas = True
except Exception:
    pass
try:
    import matplotlib.pyplot as plt
    have_matplotlib = True
except Exception:
    pass

# Criar pasta para salvar resultados
out_dir = Path('../../data') / 'embeddings'
out_dir.mkdir(parents=True, exist_ok=True)

# Montar linhas para CSV
rows = []
for k, v in results.items():
    try:
        sim_12 = cosine_sim(v['embeddings'][0], v['embeddings'][1])
        sim_13 = cosine_sim(v['embeddings'][0], v['embeddings'][2])
    except Exception:
        sim_12 = None
        sim_13 = None
    
    # Normalizar tokens (somar se for lista)
    tokens_value = v.get('tokens')
    if isinstance(tokens_value, list):
        tokens_value = sum(t for t in tokens_value if t is not None)

    # Mapear provider e método (para comparação direta OpenAI/Google)
    if k.startswith('openai'):
        provider = 'OpenAI'
        method = 'SDK'  # usamos o SDK OpenAI (openai python client)
    elif k == 'google':
        provider = 'Google'
        method = 'REST'  # usamos REST API para token count
    else:
        provider = 'Other'
        method = 'Unknown'
    
    rows.append({
        'model': k,
        'provider': provider,
        'method': method,
        'dim': v.get('dim'),
        'tokens': tokens_value,
        'sim_12': sim_12,
        'sim_13': sim_13,
        'est_cost_usd': results_cost.get(k)
    })

csv_path = out_dir / 'comparative_results.csv'
fieldnames = ['model', 'provider', 'method', 'dim', 'tokens', 'sim_12', 'sim_13', 'est_cost_usd']
with open(csv_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for r in rows:
        writer.writerow(r)
# If pandas available, show DataFrame
if have_pandas and pd is not None:
    df = pd.DataFrame(rows)
    
    # Reorder columns to show provider/method first
    df = df[fieldnames]
    
    # Configurar pandas para exibir números com mais casas decimais sem notação científica
    pd.set_option('display.float_format', lambda x: f'{x:.8f}')
    
    print('\nDataFrame summary:')
    display(df)
else:
    print('\nCSV content preview:')
    for r in rows:
        print(r)

# Tentar criar plots se matplotlib estiver disponível
if have_matplotlib and plt is not None:
    # Make arrays for plotting
    models = [r['model'] for r in rows]
    sim12 = [r['sim_12'] if r['sim_12'] is not None else 0 for r in rows]
    sim13 = [r['sim_13'] if r['sim_13'] is not None else 0 for r in rows]
    costs = [r['est_cost_usd'] if r['est_cost_usd'] is not None else 0 for r in rows]

    # Similarity bar chart
    plt.figure(figsize=(8, 4))
    x = range(len(models))
    plt.bar(x, sim12, width=0.4, label='Sim 1-2')
    plt.bar([i + 0.4 for i in x], sim13, width=0.4, label='Sim 1-3')
    plt.title('Similaridade (cosine) por modelo')
    plt.xticks([i + 0.2 for i in x], models)
    plt.ylim(0, 1)
    plt.legend()
    plt.tight_layout()
    plot1 = out_dir / 'similarity_comparison.png'
    plt.savefig(plot1)
    print(f'Saved plot to {plot1.resolve()}')
    plt.close()

    # Cost chart
    plt.figure(figsize=(6, 4))
    plt.bar(models, costs, color='orange')
    plt.title('Estimated cost per run (USD)')
    plt.ylabel('USD')
    plt.tight_layout()
    plot2 = out_dir / 'cost_comparison.png'
    plt.savefig(plot2)
    print(f'Saved plot to {plot2.resolve()}')
    plt.close()
else:
    print('\nMatplotlib or pandas not installed; plots skipped. To enable plots: pip install pandas matplotlib')

# Instruções para exportar o notebook em slides/PDF
print('\nInstruções para exportar o notebook como slides (Reveal.js) ou PDF:')
print('1) Para gerar slides HTML:')
print("   jupyter nbconvert --to slides notebooks/quick_test_classroom.ipynb --reveal-prefix 'https://cdnjs.cloudflare.com/ajax/libs/reveal.js/4.3.1/'")
print('2) Para gerar PDF (pode exigir LaTeX):')
print('   jupyter nbconvert --to pdf notebooks/quick_test_classroom.ipynb')



DataFrame summary:


,model,provider,method,dim,tokens,sim_12,sim_13,est_cost_usd
0,openai_small,OpenAI,SDK,1536,32,0.81346465,0.14203189,0.00000064
1,openai_large,OpenAI,SDK,3072,32,0.84903881,0.23315865,0.00000416
2,google,Google,REST,3072,24,0.88463019,0.53127521,0.00000360


Saved plot to E:\01-projetos\11-work\11.34-engenharia-vetorial\data\embeddings\similarity_comparison.png
Saved plot to E:\01-projetos\11-work\11.34-engenharia-vetorial\data\embeddings\cost_comparison.png

Instruções para exportar o notebook como slides (Reveal.js) ou PDF:
1) Para gerar slides HTML:
   jupyter nbconvert --to slides notebooks/quick_test_classroom.ipynb --reveal-prefix 'https://cdnjs.cloudflare.com/ajax/libs/reveal.js/4.3.1/'
2) Para gerar PDF (pode exigir LaTeX):
   jupyter nbconvert --to pdf notebooks/quick_test_classroom.ipynb


## 🔍 Novas colunas adicionadas (Provider, Method, Tokens, Est Cost USD)

Para facilitar comparações entre **uso direto de APIs (Google/OpenAI)** e outras abordagens, adicionamos as colunas abaixo ao relatório e ao CSV:

- **Provider**: Provedor do embedding (ex: OpenAI, Google)  
- **Method**: 'REST' para chamadas REST diretas, 'SDK' para clientes/SDKs, 'Other' para alternativos  
- **Tokens**: Contagem exata de tokens retornada pela API (ou soma quando múltiplas chamadas)  
- **Est Cost USD**: Estimativa de custo por execução com base em token count e preços configurados no `PRICING`

Essas colunas ajudam a comparar não só a qualidade do embedding (similaridade/diferença) mas também o custo e a forma de integração (SDK vs REST).  

> Observação: mantivemos todas as colunas existentes (`Modelo`, `Dimensões`, `Tempo Médio`, `Sim 1-2`, `Sim 1-3`, `Diferença`) — apenas acrescentamos novos dados para análise.